In [7]:
import pandas as pd
from mockcreator import MockCreator

In [8]:
df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5], # unique
    'name': ['Alice', 'Bob', 'Charlie', 'Dave', 'Eve'], # specify first name provider
    'age': [25, 30, 35, 40, 45],
    'is_student': [True, True, True, False, False], # infer the weights
    'favorite_color': ['red', 'blue', 'green', 'red', 'blue'], # infer the weights 
    'height': [162, 178, 175, 180, 165] # gaussian distribution
})

# specify when to use python random package methods, instead of faker
series_creators = {
    "is_student": "random",
    "favorite_color": "random",
}
mc = MockCreator.from_dataframe(df, series_creators=series_creators)
mc.to_yaml('../metadata/inferred.yaml')

!cat ../metadata/inferred.yaml

fields:
  id:
    faker: pyint
    min_value: 1
    max_value: 5
  name:
    faker: pystr
    min_chars: 3
    max_chars: 7
  age:
    faker: pyint
    min_value: 25
    max_value: 45
  is_student:
    random: choices
    args:
    - - true
      - false
    weights:
    - 0.6
    - 0.4
  favorite_color:
    random: choices
    args:
    - - red
      - blue
      - green
    weights:
    - 0.4
    - 0.4
    - 0.2
  height:
    faker: pyint
    min_value: 162
    max_value: 180


/Users/antti/dev/openmined/PySyft/packages/mockcreator/mockcreator/mockcreator.py:64: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, series in df.iteritems():


In [9]:
mock_creator = MockCreator.from_yaml("../metadata/inferred.yaml")

# specify faker methods, overriding inferred methods
mock_creator.set_metadata("id", { "faker.unique": "pyint", "min_value": 0, "max_value": 1000 })
mock_creator.set_metadata("name", { "faker": "first_name" })

# specify gaussian distribution for height
mock_creator.set_metadata("height", { "random": "gauss", "mu": 170, "sigma": 8 })

generated = mock_creator.generate_data(10)
generated

,id,name,age,is_student,favorite_color,height
0,499,Lauren,25,True,red,169.020505
1,394,Teresa,36,False,red,171.845906
2,510,Crystal,37,False,red,163.801852
3,780,Alexander,41,True,red,166.701847
4,919,Eric,41,False,blue,172.316592
5,174,George,31,True,blue,169.558594
6,878,Beth,40,False,blue,148.301706
7,446,Matthew,26,False,red,168.819179
8,216,Michael,31,True,red,178.270392
9,359,Sarah,30,True,green,186.121346


In [10]:
generated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              10 non-null     int64  
 1   name            10 non-null     object 
 2   age             10 non-null     int64  
 3   is_student      10 non-null     bool   
 4   favorite_color  10 non-null     object 
 5   height          10 non-null     float64
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 538.0+ bytes


In [12]:
mock_creator.to_yaml("../metadata/final.yaml")
!cat ../metadata/final.yaml

fields:
  id:
    faker.unique: pyint
    min_value: 0
    max_value: 1000
  name:
    faker: first_name
  age:
    faker: pyint
    min_value: 25
    max_value: 45
  is_student:
    random: choices
    args:
    - - true
      - false
    weights:
    - 0.6
    - 0.4
  favorite_color:
    random: choices
    args:
    - - red
      - blue
      - green
    weights:
    - 0.4
    - 0.4
    - 0.2
  height:
    random: gauss
    mu: 170
    sigma: 8
